## Neural Network

In [496]:
import numpy as np
import polars as pl

df = pl.read_csv("dengue_features_train.csv")
labels = pl.read_csv("dengue_labels_train.csv")
df.head()

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
str,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""sj""",1990,18,"""1990-04-30""",0.1226,0.103725,0.1984833,0.1776167,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.0,73.365714,12.42,14.012857,2.628571,25.442857,6.9,29.4,20.0,16.0
"""sj""",1990,19,"""1990-05-07""",0.1699,0.142175,0.1623571,0.1554857,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
"""sj""",1990,20,"""1990-05-14""",0.03225,0.1729667,0.1572,0.1708429,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.1,82.052857,34.54,16.848571,2.3,26.714286,6.485714,32.2,22.8,41.4
"""sj""",1990,21,"""1990-05-21""",0.1286333,0.2450667,0.2275571,0.2358857,15.36,298.987143,299.228571,295.31,301.4,297.0,13.9,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
"""sj""",1990,22,"""1990-05-28""",0.1962,0.2622,0.2512,0.24734,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.2,80.46,7.52,17.21,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [497]:
labels.head()

city,year,weekofyear,total_cases
str,i64,i64,i64
"""sj""",1990,18,4
"""sj""",1990,19,5
"""sj""",1990,20,4
"""sj""",1990,21,3
"""sj""",1990,22,6


In [498]:
df.null_count()

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,194,52,22,22,13,10,10,10,10,10,10,10,13,10,10,43,43,20,14,22


In [499]:
pixel_cols = ["ndvi_ne", "ndvi_nw", "ndvi_se", "ndvi_sw"]

df_full = df.with_columns(
    pl.col(pixel_cols)
    .fill_null(pl.col(pixel_cols).mean().over(["city", "weekofyear"]))
    .fill_null(pl.col(pixel_cols).mean().over(["city"]))
    .fill_null(pl.col(pixel_cols).mean())
)

In [500]:
df_full.null_count()

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,13,10,10,10,10,10,10,10,13,10,10,43,43,20,14,22


In [501]:
all_weather_cols = [
    "ndvi_ne", "ndvi_nw", "ndvi_se", "ndvi_sw",
    "station_max_temp_c", "station_min_temp_c", "station_avg_temp_c",
    "station_precip_mm", "station_diur_temp_rng_c", "station_avg_temp_c", 
    "reanalysis_tdtr_k", "reanalysis_specific_humidity_g_per_kg", "reanalysis_sat_precip_amt_mm", 
    "reanalysis_relative_humidity_percent", "reanalysis_precip_amt_kg_per_m2", "reanalysis_min_air_temp_k",
    "reanalysis_max_air_temp_k", "reanalysis_dew_point_temp_k", "reanalysis_avg_temp_k", "reanalysis_air_temp_k",
    "precipitation_amt_mm"
]

kelvin_cols = [
    "reanalysis_min_air_temp_k",
    "reanalysis_max_air_temp_k",
    "reanalysis_dew_point_temp_k",
    "reanalysis_avg_temp_k",
    "reanalysis_air_temp_k"
]

df_full = df_full.with_columns(
    pl.col(all_weather_cols)
    .fill_null(pl.col(all_weather_cols).mean().over(["city", "weekofyear"]))
    .fill_null(pl.col(all_weather_cols).mean().over(["city"]))
    .fill_null(pl.col(all_weather_cols).mean())
)

df_full = df_full.with_columns(
    (pl.col(kelvin_cols) - 273.15).name.suffix("_celsius")
)

df_full = df_full.drop(kelvin_cols)

In [502]:
df_full.null_count()

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,reanalysis_min_air_temp_k_celsius,reanalysis_max_air_temp_k_celsius,reanalysis_dew_point_temp_k_celsius,reanalysis_avg_temp_k_celsius,reanalysis_air_temp_k_celsius
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [503]:
df_full.head()

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,reanalysis_min_air_temp_k_celsius,reanalysis_max_air_temp_k_celsius,reanalysis_dew_point_temp_k_celsius,reanalysis_avg_temp_k_celsius,reanalysis_air_temp_k_celsius
str,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""sj""",1990,18,"""1990-04-30""",0.1226,0.103725,0.1984833,0.1776167,12.42,32.0,73.365714,12.42,14.012857,2.628571,25.442857,6.9,29.4,20.0,16.0,22.75,26.65,19.264286,24.592857,24.422857
"""sj""",1990,19,"""1990-05-07""",0.1699,0.142175,0.1623571,0.1554857,22.82,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,23.25,27.75,20.801429,25.292857,25.061429
"""sj""",1990,20,"""1990-05-14""",0.03225,0.1729667,0.1572,0.1708429,34.54,26.1,82.052857,34.54,16.848571,2.3,26.714286,6.485714,32.2,22.8,41.4,24.15,27.35,22.284286,25.728571,25.631429
"""sj""",1990,21,"""1990-05-21""",0.1286333,0.2450667,0.2275571,0.2358857,15.36,13.9,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,23.85,28.25,22.16,26.078571,25.837143
"""sj""",1990,22,"""1990-05-28""",0.1962,0.2622,0.2512,0.24734,7.52,12.2,80.46,7.52,17.21,3.014286,28.942857,9.371429,35.0,23.9,5.8,24.35,28.75,22.671429,26.514286,26.368571


# Cyclic encoding for weeks

In [504]:
# https://towardsdatascience.com/cyclical-encoding-an-alternative-to-one-hot-encoding-for-time-series-features-4db46248ebba/

df_full = df_full.with_columns([
    (2 * np.pi * pl.col("weekofyear") / 52).sin().alias("week_sin"),
    (2 * np.pi * pl.col("weekofyear") / 52).cos().alias("week_cos")
])

In [505]:
df_full.head()

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,reanalysis_min_air_temp_k_celsius,reanalysis_max_air_temp_k_celsius,reanalysis_dew_point_temp_k_celsius,reanalysis_avg_temp_k_celsius,reanalysis_air_temp_k_celsius,week_sin,week_cos
str,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""sj""",1990,18,"""1990-04-30""",0.1226,0.103725,0.1984833,0.1776167,12.42,32.0,73.365714,12.42,14.012857,2.628571,25.442857,6.9,29.4,20.0,16.0,22.75,26.65,19.264286,24.592857,24.422857,0.822984,-0.568065
"""sj""",1990,19,"""1990-05-07""",0.1699,0.142175,0.1623571,0.1554857,22.82,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,23.25,27.75,20.801429,25.292857,25.061429,0.748511,-0.663123
"""sj""",1990,20,"""1990-05-14""",0.03225,0.1729667,0.1572,0.1708429,34.54,26.1,82.052857,34.54,16.848571,2.3,26.714286,6.485714,32.2,22.8,41.4,24.15,27.35,22.284286,25.728571,25.631429,0.663123,-0.748511
"""sj""",1990,21,"""1990-05-21""",0.1286333,0.2450667,0.2275571,0.2358857,15.36,13.9,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,23.85,28.25,22.16,26.078571,25.837143,0.568065,-0.822984
"""sj""",1990,22,"""1990-05-28""",0.1962,0.2622,0.2512,0.24734,7.52,12.2,80.46,7.52,17.21,3.014286,28.942857,9.371429,35.0,23.9,5.8,24.35,28.75,22.671429,26.514286,26.368571,0.464723,-0.885456


In [506]:
df_sj = df_full.filter(pl.col("city") == "sj")
df_iq = df_full.filter(pl.col("city") == "iq")

print(f"San Juan: {len(df_sj)} rows")
print(f"Iquitos: {len(df_iq)} rows")

San Juan: 936 rows
Iquitos: 520 rows


In [507]:
labels.head()

city,year,weekofyear,total_cases
str,i64,i64,i64
"""sj""",1990,18,4
"""sj""",1990,19,5
"""sj""",1990,20,4
"""sj""",1990,21,3
"""sj""",1990,22,6


In [508]:
exclude_cols = ["city", "year", "weekofyear", "week_start_date", "total_cases"]
feature_cols = [c for c in df_full.columns if c not in exclude_cols]

labels_sj = labels.filter(pl.col("city") == "sj")
labels_iq = labels.filter(pl.col("city") == "iq")

In [509]:

X_sj = df_sj.select(feature_cols).to_numpy().astype(np.float32)
X_iq = df_iq.select(feature_cols).to_numpy().astype(np.float32)

y_sj = labels_sj["total_cases"].to_numpy().astype(np.float32)
y_iq = labels_iq["total_cases"].to_numpy().astype(np.float32)


In [510]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train_sj, X_val_sj, y_train_sj, y_val_sj = train_test_split(X_sj, y_sj, test_size=0.2, random_state=42)
scaler_sj = StandardScaler()
X_train_sj = scaler_sj.fit_transform(X_train_sj)
X_val_sj = scaler_sj.transform(X_val_sj)

X_train_iq, X_val_iq, y_train_iq, y_val_iq = train_test_split(X_iq, y_iq, test_size=0.2, random_state=42)
scaler_iq = StandardScaler()
X_train_iq = scaler_iq.fit_transform(X_train_iq)
X_val_iq = scaler_iq.transform(X_val_iq)

In [511]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# San Juan
train_ds_sj = TensorDataset(torch.tensor(X_train_sj), torch.tensor(y_train_sj).unsqueeze(1))
train_loader_sj = DataLoader(train_ds_sj, batch_size=32, shuffle=True)

# Iquitos
train_ds_iq = TensorDataset(torch.tensor(X_train_iq), torch.tensor(y_train_iq).unsqueeze(1))
train_loader_iq = DataLoader(train_ds_iq, batch_size=32, shuffle=True)

In [512]:
class DengueNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.net(x)

model = DengueNN(X_train_scaled.shape[1])

In [513]:
# San Juan model
model_sj = DengueNN(X_train_sj.shape[1])

# Iquitos model
model_iq = DengueNN(X_train_iq.shape[1])

In [514]:
optimizer_sj = torch.optim.Adam(model_sj.parameters(), lr=0.001)
loss_func = nn.L1Loss()

print("Training San Juan model")
for epoch in range(80):
    model_sj.train()
    for X_batch, y_batch in train_loader_sj:
        optimizer_sj.zero_grad()
        pred = model_sj(X_batch)
        loss = loss_func(pred, y_batch)
        loss.backward()
        optimizer_sj.step()
    
    if (epoch + 1) % 10 == 0:
        model_sj.eval()
        with torch.no_grad():
            val_preds = model_sj(torch.tensor(X_val_sj))
            val_loss = loss_func(val_preds, torch.tensor(y_val_sj).unsqueeze(1))
        print(f"Epoch {epoch+1}, Val MSE: {val_loss.item():.2f}")

optimizer_iq = torch.optim.Adam(model_iq.parameters(), lr=0.001)

print("\nTraining Iquitos model")
for epoch in range(100):
    model_iq.train()
    for X_batch, y_batch in train_loader_iq:
        optimizer_iq.zero_grad()
        pred = model_iq(X_batch)
        loss = loss_func(pred, y_batch)
        loss.backward()
        optimizer_iq.step()
    
    if (epoch + 1) % 10 == 0:
        model_iq.eval()
        with torch.no_grad():
            val_preds = model_iq(torch.tensor(X_val_iq))
            val_loss = loss_func(val_preds, torch.tensor(y_val_iq).unsqueeze(1))
        print(f"Epoch {epoch+1}, Val MAE: {val_loss.item():.2f}")

Training San Juan model
Epoch 10, Val MSE: 22.11
Epoch 20, Val MSE: 20.84
Epoch 30, Val MSE: 20.58
Epoch 40, Val MSE: 20.44
Epoch 50, Val MSE: 20.41
Epoch 60, Val MSE: 20.40
Epoch 70, Val MSE: 20.37
Epoch 80, Val MSE: 20.50

Training Iquitos model
Epoch 10, Val MAE: 5.44
Epoch 20, Val MAE: 5.52
Epoch 30, Val MAE: 5.61
Epoch 40, Val MAE: 5.63
Epoch 50, Val MAE: 5.66
Epoch 60, Val MAE: 5.69
Epoch 70, Val MAE: 5.81
Epoch 80, Val MAE: 5.70
Epoch 90, Val MAE: 5.65
Epoch 100, Val MAE: 5.71


# Linear Regression

In [519]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Train San Juan model
lr_sj = LinearRegression()
lr_sj.fit(X_train_sj, y_train_sj)

# Train Iquitos model
lr_iq = LinearRegression()
lr_iq.fit(X_train_iq, y_train_iq)

# Evaluate
preds_sj = lr_sj.predict(X_val_sj)
preds_iq = lr_iq.predict(X_val_iq)

print(f"San Juan MAE: {mean_absolute_error(y_val_sj, preds_sj):.2f}")
print(f"Iquitos MAE: {mean_absolute_error(y_val_iq, preds_iq):.2f}")

San Juan MAE: 23.88
Iquitos MAE: 6.96


In [515]:
from sklearn.metrics import mean_absolute_error

# San Juan
model_sj.eval()
with torch.no_grad():
    preds_sj = model_sj(torch.tensor(X_val_sj)).numpy().flatten()
print(f"San Juan MAE: {mean_absolute_error(y_val_sj, preds_sj):.2f}")

# Iquitos
model_iq.eval()
with torch.no_grad():
    preds_iq = model_iq(torch.tensor(X_val_iq)).numpy().flatten()
print(f"Iquitos MAE: {mean_absolute_error(y_val_iq, preds_iq):.2f}")

San Juan MAE: 20.50
Iquitos MAE: 5.71


## Test

In [516]:
test_df = pl.read_csv('dengue_features_test.csv')

pixel_cols = ["ndvi_ne", "ndvi_nw", "ndvi_se", "ndvi_sw"]

test_df = test_df.with_columns(
    pl.col(pixel_cols)
    .fill_null(pl.col(pixel_cols).mean().over(["city", "weekofyear"]))
    .fill_null(pl.col(pixel_cols).mean().over(["city"]))
    .fill_null(pl.col(pixel_cols).mean())
)

all_weather_cols = [
    "ndvi_ne", "ndvi_nw", "ndvi_se", "ndvi_sw",
    "station_max_temp_c", "station_min_temp_c", "station_avg_temp_c",
    "station_precip_mm", "station_diur_temp_rng_c", "station_avg_temp_c",
    "reanalysis_tdtr_k", "reanalysis_specific_humidity_g_per_kg", "reanalysis_sat_precip_amt_mm",
    "reanalysis_relative_humidity_percent", "reanalysis_precip_amt_kg_per_m2", "reanalysis_min_air_temp_k",
    "reanalysis_max_air_temp_k", "reanalysis_dew_point_temp_k", "reanalysis_avg_temp_k", "reanalysis_air_temp_k",
    "precipitation_amt_mm"
]

kelvin_cols = [
    "reanalysis_min_air_temp_k",
    "reanalysis_max_air_temp_k",
    "reanalysis_dew_point_temp_k",
    "reanalysis_avg_temp_k",
    "reanalysis_air_temp_k"
]

test_df = test_df.with_columns(
    pl.col(all_weather_cols)
    .fill_null(pl.col(all_weather_cols).mean().over(["city", "weekofyear"]))
    .fill_null(pl.col(all_weather_cols).mean().over(["city"]))
    .fill_null(pl.col(all_weather_cols).mean())
)

test_df = test_df.with_columns(
    (pl.col(kelvin_cols) - 273.15).name.suffix("_celsius")
)

test_df = test_df.drop(kelvin_cols)

test_df = test_df.with_columns([
    (2 * np.pi * pl.col("weekofyear") / 52).sin().alias("week_sin"),
    (2 * np.pi * pl.col("weekofyear") / 52).cos().alias("week_cos")
])

test_sj = test_df.filter(pl.col("city") == "sj")
test_iq = test_df.filter(pl.col("city") == "iq")

X_test_sj = test_sj.select(feature_cols).to_numpy().astype(np.float32)
X_test_iq = test_iq.select(feature_cols).to_numpy().astype(np.float32)

X_test_sj_scaled = scaler_sj.transform(X_test_sj)
X_test_iq_scaled = scaler_iq.transform(X_test_iq)

ids_sj = test_sj.select(["city", "year", "weekofyear"])
ids_iq = test_iq.select(["city", "year", "weekofyear"])

In [517]:
model_sj.eval()
model_iq.eval()

with torch.no_grad():
    preds_sj = model_sj(torch.tensor(X_test_sj_scaled)).numpy().flatten()
    preds_iq = model_iq(torch.tensor(X_test_iq_scaled)).numpy().flatten()

preds_sj = np.clip(np.round(preds_sj), 0, None).astype(int)
preds_iq = np.clip(np.round(preds_iq), 0, None).astype(int)

results_sj = ids_sj.to_pandas()
results_sj['total_cases'] = preds_sj

results_iq = ids_iq.to_pandas()
results_iq['total_cases'] = preds_iq

results = pd.concat([results_sj, results_iq])
results.to_csv('predictions.csv', index=False)

print(f"Saved {len(results)} predictions")
print("Sample predictions:")
print(results.head())

Saved 416 predictions
Sample predictions:
  city  year  weekofyear  total_cases
0   sj  2008          18            3
1   sj  2008          19            4
2   sj  2008          20           12
3   sj  2008          21           23
4   sj  2008          22           14


In [518]:
print(f"Scaler expects: {scaler_sj.n_features_in_} features")
print(f"Test has: {X_test_sj.shape[1]} features")

Scaler expects: 22 features
Test has: 22 features


# LinearRegression predictions

In [521]:
test = pd.read_csv('dengue_features_test.csv')

test_sj = test[test['city'] == 'sj']
test_iq = test[test['city'] == 'iq']

ids_sj = test_sj[['city', 'year', 'weekofyear']]
ids_iq = test_iq[['city', 'year', 'weekofyear']]

drop_cols = ['city', 'year', 'weekofyear', 'week_start_date']
X_test_sj = test_sj.drop(columns=drop_cols).fillna(0).to_numpy()
X_test_iq = test_iq.drop(columns=drop_cols).fillna(0).to_numpy()


preds_sj = lr_sj.predict(X_test_sj_scaled)
preds_iq = lr_iq.predict(X_test_iq_scaled)


preds_sj = np.clip(np.round(preds_sj), 0, None).astype(int)
preds_iq = np.clip(np.round(preds_iq), 0, None).astype(int)

results_sj = ids_sj.copy()
results_sj['total_cases'] = preds_sj

results_iq = ids_iq.copy()
results_iq['total_cases'] = preds_iq

results = pd.concat([results_sj, results_iq])
results.to_csv('predictions_lr.csv', index=False)

print(f"Saved {len(results)} predictions")

Saved 416 predictions
